In [22]:
import pandas as pd
import numpy as np
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
import ast
from scipy.sparse import hstack

In [93]:
movies = pd.read_csv("../../Data/serendipity-sac2018/movies.csv", error_bad_lines=False)
imdbData = pd.read_csv("../Intermediate_data/IMDb_cldata.csv")
imdbData.drop("Unnamed: 0", axis=1, inplace=True)
movies.head()

Skipping line 19833: expected 8 fields, saw 10
Skipping line 34143: expected 8 fields, saw 9
Skipping line 36015: expected 8 fields, saw 10
Skipping line 37260: expected 8 fields, saw 12
Skipping line 44379: expected 8 fields, saw 10
Skipping line 47551: expected 8 fields, saw 10



,movieId,title,releaseDate,directedBy,starring,imdbId,tmdbId,genres
0,1,Toy Story (1995),1995-11-19,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",114709,862.0,"Adventure,Animation,Children,Comedy,Fantasy"
1,2,Jumanji (1995),1995-12-15,Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",113497,8844.0,"Adventure,Children,Fantasy"
2,3,Grumpier Old Men (1995),1995-01-01,Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret , Sop...",113228,15602.0,"Comedy,Romance"
3,4,Waiting to Exhale (1995),1996-01-15,Forest Whitaker,"Angela Bassett, Loretta Devine, Whitney Housto...",114885,31357.0,"Comedy,Drama,Romance"
4,5,Father of the Bride Part II (1995),1995-12-08,Charles Shyer,"Steve Martin, Martin Short, Diane Keaton, Kimb...",113041,11862.0,Comedy


In [94]:
allMovies = movies[["movieId", "imdbId"]].merge(imdbData, on = "imdbId", how = "left")
allMovies.head()

,movieId,imdbId,plot,rating,runtimes,votes,country codes_us,country codes_gb,country codes_fr,country codes_it,...,certificates_G,certificates_NR,certificates_PG,certificates_R,kind_movie,kind_series,year_1800s,year_1900s,year_1950s,year_2000s
0,1,114709,"[u""A cowboy doll is profoundly threatened and ...",8.3,81.0,837115.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,2,113497,[u'When two kids find and play a magical board...,7.0,104.0,287007.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,3,113228,"[u""John and Max resolve to save their beloved ...",6.7,101.0,23107.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,4,114885,"[u""Based on Terry McMillan's novel, this film ...",5.9,124.0,9034.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,5,113041,[u'George Banks must deal not only with the pr...,6.0,106.0,32568.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [98]:
allMovies.columns = [col.replace(" ","_") for col in allMovies.columns]
allMovies.columns

Index([u'movieId', u'imdbId', u'plot', u'rating', u'runtimes', u'votes',
       u'country_codes_us', u'country_codes_gb', u'country_codes_fr',
       u'country_codes_it', u'country_codes_ca', u'country_codes_de',
       u'country_codes_jp', u'country_codes_es', u'country_codes_in',
       u'country_codes_hk', u'genres_Drama', u'genres_Comedy',
       u'genres_Thriller', u'genres_Romance', u'genres_Action',
       u'genres_Crime', u'genres_Horror', u'genres_Adventure',
       u'genres_Documentary', u'genres_Mystery', u'language_codes_en',
       u'language_codes_fr', u'language_codes_es', u'language_codes_it',
       u'language_codes_de', u'language_codes_ja', u'language_codes_ru',
       u'language_codes_cmn', u'language_codes_hi', u'language_codes_sv',
       u'certificates_G', u'certificates_NR', u'certificates_PG',
       u'certificates_R', u'kind_movie', u'kind_series', u'year_1800s',
       u'year_1900s', u'year_1950s', u'year_2000s'],
      dtype='object')

In [75]:
train = pd.read_csv("../Intermediate_data/train_subset.csv")
train.drop("Unnamed: 0", axis=1, inplace=True)
train.head()

,userId,movieId,rating,timestamp,year
0,127137,115824,4.5,2017-12-31 23:59:50,2017
1,142748,2762,5.0,2017-12-31 23:59:41,2017
2,196198,1198,3.5,2017-12-31 23:59:17,2017
3,142748,71057,4.5,2017-12-31 23:58:09,2017
4,142748,152077,5.0,2017-12-31 23:57:32,2017


In [96]:
colsToScale = ["runtimes", "votes"]

scaler = MinMaxScaler()
allMovies[colsToScale] = scaler.fit_transform(allMovies[colsToScale])


In [223]:
def getSimilarityScore(movie1, movie2, similarFeatures, dissimilarFeatures):
#     d1 = np.linalg.norm(movie1[similarFeatures] - movie2[similarFeatures], ord=1)
#     d2 = np.linalg.norm(movie1[dissimilarFeatures] - movie2[dissimilarFeatures], ord=1)

    d1 = np.linalg.norm(movie1[:len(similarFeatures)] - movie2[:len(similarFeatures)], ord=1)
    d2 = np.linalg.norm(movie1[len(similarFeatures):] - movie2[len(similarFeatures):], ord=1)
    
    ## To avoid division by 0
    d2 += 1e-3
    similarityScore = (d1 + (1/d2))
#     similarityScore = (d2 + (1/d1))
    return similarityScore

def setupSimilarityMatirx(likedMovies, allMovies, similarFeatures, dissimilarFeatures):
    similarityMatrix = np.zeros(shape=(len(likedMovies), len(allMovies)))
    data1 = np.array(likedMovies[similarFeatures + dissimilarFeatures])
    data2 = np.array(allMovies[similarFeatures + dissimilarFeatures])
    for i in range(len(likedMovies)):
        if i%20 == 0:
            print(str(i) + "/" + str(len(likedMovies)))
        similarityScores = []
        for j in range(len(allMovies)):
            similarityScores.append(getSimilarityScore(data1[i,:], data2[j,:], 
                                                       similarFeatures, dissimilarFeatures))
#             similarityScores.append(getSimilarityScore(likedMovies.iloc[i], allMovies.iloc[j], 
#                                                        similarFeatures, dissimilarFeatures))
                
        similarityMatrix[i,:] = similarityScores
    return similarityMatrix

def getRelevantMovies(likedMovies, similarityMatrix, top_n = 10):
    indices = similarityMatrix.ravel().argsort()
    indices = indices[-top_n:]%similarityMatrix.shape[1]
    return np.unique(indices)

def getLikedMovies(userMovieRatings, allMovies, ratingThreshold = 3.5, numMoviesToReturn = 10):
    
    likedMovieIds = userMovieRatings.loc[userMovieRatings["rating"] >= ratingThreshold, "movieId"]
    likedMovies = allMovies[allMovies["movieId"].apply(lambda x: x in np.unique(likedMovieIds))]
    return likedMovies.head(numMoviesToReturn)

def getPredictions(userMovieRatings, allMovies):
    similarFeatures = ["runtimes", "votes"]
    dissimilarFeatures = ["country_codes_us", "country_codes_gb", "country_codes_fr", "country_codes_ca"]
    likedMovies = getLikedMovies(userMovieRatings, allMovies)
    print(likedMovies.head())
    print(len(likedMovies))
    similarityMatrix = setupSimilarityMatirx(likedMovies, allMovies, 
                                             similarFeatures, dissimilarFeatures)
    predictionIndices = getRelevantMovies(likedMovies, similarityMatrix)
    
    return allMovies.loc[predictionIndices,:], likedMovies

In [ ]:
user1 = train[train["userId"] == 127137]
predictions, likedMovies = getPredictions(user1,allMovies)

       movieId  imdbId                                               plot  \
5815      6016  317248  [u"In the slums of Rio, two kids' paths diverg...   
20142    96606  770802  [u'Filmed over nearly five years in twenty-fiv...   
13325    60069  910970  [u'In the distant future, a small waste-collec...   
9587     27773  364569  [u'After being kidnapped and imprisoned for fi...   
11228    44555  405094  [u'In 1984 East Berlin, an agent of the secret...   

       rating  runtimes     votes  country_codes_us  country_codes_gb  \
5815      8.6  0.102789  0.304348               0.0               0.0   
20142     8.5  0.080478  0.014182               1.0               0.0   
13325     8.4  0.077291  0.431184               0.0               0.0   
9587      8.4  0.094821  0.219481               1.0               0.0   
11228     8.4  0.108367  0.153799               1.0               0.0   

       country_codes_fr  country_codes_it  ...  certificates_G  \
5815                0.0         